In [1]:
import pandas as pd
import numpy as np
import mne
import mne_bids
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

import q1k_init_tools as qit

import glob

import warnings
warnings.filterwarnings('ignore')


In [ ]:
project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
task_id_in = "ap"
task_id_in_et = "ap"
task_id_out = "as"
subject_id = '002'
session_id = '01'
run_id = '1'

session_path_eeg = project_path + 'sourcefiles/' + subject_id + '_' + session_id[1] + '/' + subject_id + '_' + session_id[1] + '_eeg/'
session_file_name_eeg = glob.glob(session_path_eeg + '*_' + task_id_in + '_*.mff')

session_path_et = project_path + 'sourcefiles/' + subject_id + '_' + session_id[1] + '/' + subject_id + '_' + session_id[1] + '_et/'
session_file_name_et = glob.glob(session_path_et + '*_' + task_id_in_et + '_*.asc')


if task_id_out == 'ap':
    event_dict_offset = 1
if task_id_out == 'as':
    event_dict_offset = 1
    din_str = ('DIN2','DIN2')
if task_id_out == 'go':
    event_dict_offset = 1
    din_str = ('DIN2','DIN2')
if task_id_out == 'mn':
    event_dict_offset = 1
if task_id_out == 'vp':
    event_dict_offset = 0
    din_str = ('DIN2','DIN3')

In [ ]:
print(session_path_eeg)
print(session_file_name_eeg[0])
raw = mne.io.read_raw_egi(session_file_name_eeg[0])

In [ ]:
eeg_events = mne.find_events(raw, shortest_event = 1)
#eeg_events = mne.find_events(raw, stim_channel="STI 014", shortest_event = 1)
eeg_event_dict = qit.get_event_dict(raw, eeg_events, event_dict_offset)

In [ ]:
eeg_event_dict

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='Original event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')
py.iplot(fig)


In [ ]:
din_diffs = []
din_diffs_time = []
last_din = 0
# Iterate over the array
for row in eeg_events:
    # Check if the third column corresponds to e1 or e2 values
    if row[2] == eeg_event_dict[din_str[0]] or row[2] == eeg_event_dict[din_str[0]]:
        # Calculate the sequential difference between values in the first column
        if last_din > 0:
            din_diffs.append(row[0] - last_din)
            din_diffs_time.append(row[0])
            last_din = row[0]
        else:
            last_din = row[0]
            #print(last_din)


In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=din_diffs_time, y=din_diffs)
fig.update_layout(title='Time between DIN events')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Inter event interval')

In [ ]:
# handle task specific EEG event interpretation..
eeg_events, eeg_stims, eeg_iti, eeg_din_offset, eeg_event_dict, new_events = qit.eeg_event_test(eeg_events, eeg_event_dict, din_str, task_name=task_id_out)

In [ ]:
column_values = eeg_events[:, 2]

# Get unique values and their counts
unique_values, counts = np.unique(column_values, return_counts=True)

# Print the counts of each unique value
print("Counts of each event ID value:")
for value, count in zip(unique_values, counts):
    print(f"Event ID: {value}, Count: {count}")

# Print the event labels
print("Label\tValue")
for label, value in eeg_event_dict.items():
    print(f"{label}\t{value}")

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='DIN updated event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')

fig.show()

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_stims[1:,0],y=eeg_iti)
fig.update_layout(title='Stim DIN event Inter Trial Intervals (ITI)')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Stim DIBN event ITI (ms)')
fig.show()

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_stims[:,0],y=eeg_din_offset)
fig.update_layout(title='Stim DIN offsets')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Stim DIN offset (ms)')

fig.show()

In [ ]:
fig = raw.plot_sensors(show_names=True)

In [ ]:
# specify power line frequency as required by BIDS
raw.info["line_freq"] = 60

bids_path = mne_bids.BIDSPath(
    subject=subject_id, session=session_id, task=task_id_out, run="1", datatype="eeg", root=project_path
)

print(bids_path)
mne_bids.write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events=eeg_events,
    event_id=eeg_event_dict,
    format = "EDF",
    overwrite=True,
)

# The following code blocks are for eyelink syncing...

In [ ]:
#read the asc eye tracking data and convert it to a dataframe...
et_raw = mne.io.read_raw_eyelink(session_file_name_et[0])
et_raw_fresh=et_raw.copy() #make a fresh copy for later
et_raw_df = et_raw.to_data_frame()

#get the events from the annotation structure
et_annot_events, et_annot_event_dict = mne.events_from_annotations(et_raw)


In [ ]:
et_annot_event_dict

In [ ]:
# peak... at the ET event scatter plot.. event time stamp by label index
fig=px.scatter(x=et_annot_events[:,0],y=et_annot_events[:,2])
fig.update_layout(title='Original event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')
py.iplot(fig)

In [ ]:
#handle task specific ET event interpretation..
et_event_raw_df, et_events, et_stims, et_iti = qit.et_event_test(et_raw_df,task_name=task_id_in_et)

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=et_stims.index[1:],y=et_iti[1:])
fig.update_layout(title='ET Stim DIN event Inter Trial Intervals (ITI)')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Stim DIBN event ITI (ms)')
fig.show()

In [ ]:
# peak... at the time offset between the matched events in the EEG and ET recording 
qit.show_sync_offsets(eeg_stims,et_stims)

In [ ]:
#!jupyter nbconvert --output {"session_reports/" + bids_path.basename + ".html"} --no-input --to html q1k_eeget_init.ipynb
!jupyter nbconvert --output {"session_reports/" + bids_path.basename + ".html"} --TagRemovePreprocessor.remove_all_outputs_tags='{"exclude"}' --no-input --to html q1k_eeget_init.ipynb
#f'Total of {len(outlier_accum)} outliers out of {all_accounts.size} accounts'